# LIGO Limits

Here, we estimate the LIGO observed merger rate, given a distribution of PBH binaries. This allows us to estimate the limit placed on the PBH abundance by LIGO.

**References:** [arXiv:1602.03842](https://arxiv.org/pdf/1602.03842.pdf), [arXiv:1606.03939](https://arxiv.org/abs/1606.03939), [arXiv:1704.04628](https://arxiv.org/abs/1704.04628)

In [ ]:
%matplotlib inline

from __future__ import division

import numpy as np
import matplotlib.pyplot as pl
import matplotlib as mpl

#----- MATPLOTLIB paramaters ---------
mpl.rcParams.update({'font.size': 18,'font.family':'serif'})

mpl.rcParams['xtick.major.size'] = 7
mpl.rcParams['xtick.major.width'] = 1
mpl.rcParams['xtick.minor.size'] = 3
mpl.rcParams['xtick.minor.width'] = 1
mpl.rcParams['ytick.major.size'] = 7
mpl.rcParams['ytick.major.width'] = 1
mpl.rcParams['ytick.minor.size'] = 3
mpl.rcParams['ytick.minor.width'] = 1
#--------------------------------------

from scipy.interpolate import interp1d
from scipy.integrate import quad


### LIGO sensitivity as a function of redshift

Read in the time-volume sensitivity for LIGO, from Fig. 7 of [arXiv:1704.04628](https://arxiv.org/abs/1704.04628):

In [ ]:
sens_data_10Msun = np.loadtxt("data/LIGO_sensitivity_10.txt")
sens_data_20Msun = np.loadtxt("data/LIGO_sensitivity_20.txt")
sens_data_40Msun = np.loadtxt("data/LIGO_sensitivity_40.txt")

In [ ]:

sensitivities = [sens_data_10Msun, sens_data_20Msun, sens_data_40Msun]
labels = [r"$10\,M_\odot$",r"$20\,M_\odot$",r"$40\,M_\odot$"]

pl.figure(figsize=(7,5))

for sens, lab in zip(sensitivities, labels):
    pl.plot(sens[:,0], sens[:,1], linewidth=1.5, label=lab)
    
pl.xlim(0, 1.0)
pl.ylim(0, 0.75)

pl.xlabel(r"$z$")
pl.ylabel(r"$\mathrm{d}\langle V T \rangle/\mathrm{d}z$ (Gpc$^3$ yr)")

pl.legend(loc='upper right', frameon=False)

pl.show()

### Calculating the LIGO sensitivity

Unfortunately, we don't have sensitivity information as a function of redshift for any other PBH masses. However, the distribution of merger times turns out to be more or less uniform as a function of $t$ (and therefore $z$). We therefore only need the integrated time-volume sensitivity $\langle VT \rangle$.

We obtain this by integrating the above sensitivities as a function of redshift. For 100, 200 and 300 Solar Mass BHs, we take the 90% upper limit on the merger rate density from [arXiv:1704.04628](https://arxiv.org/abs/1704.04628). We can convert between time-volume and sensitivity using:

$$R_{90\%} = \frac{2.303}{\langle VT \rangle} \, ,$$

from Eq. 2 of that paper.

In [ ]:
M_list = np.asarray([10., 20., 40.])
VT = np.zeros_like(M_list)

#Calculate stellar mass BH sensitivities by integrating
for i, sens in enumerate(sensitivities):
    VT[i] = np.trapz(sens[:,1], sens[:,0])

#Calculate IMBH sensitivities by converting from R90    
R_90_IMBH = np.loadtxt("data/LIGO_R90_IMBH.txt")
M_list = np.append(M_list, R_90_IMBH[:,0])
VT_IMBH = np.asarray([2.303/R_90_IMBH[j,1] for j in range(3)]) 

VT = np.append(VT, VT_IMBH)

#Do some plotting
pl.figure()
pl.loglog(M_list, VT, 's')
pl.xlabel(r"$M_\mathrm{BH} \,[M_\odot]$")
pl.ylabel(r"$\langle VT \rangle \, [\mathrm{Gpc}^3 \,\mathrm{yr}]$")
pl.title("Time-volume sensitivity", fontsize=14.0)
pl.show()

### LIGO Upper limit

Estimate the LIGO upper limit on the merger rate density and save to file

In [ ]:
R90_LIGO = 2.303/VT

np.savetxt("data/R90_LIGO.txt", zip(M_list, R90_LIGO), header="90% UL on merger rate density from LIGO\nColumns: M_BH [M_solar], R_90% [Gpc^-3 yr^-1]")

M1 = np.linspace(M_list[0], M_list[-1])

lR90_interp = interp1d(np.log10(M_list), np.log10(R90_LIGO), kind='linear')

pl.figure()
pl.fill_between(M1, 10**lR90_interp(np.log10(M1)), 1e3, color='blue', alpha=0.25)
pl.loglog(M_list, R90_LIGO, 's')
pl.xlabel(r"$M_\mathrm{BH} \,[M_\odot]$")
pl.ylabel(r"$R_{90\%} \, [\mathrm{Gpc}^{-3} \,\mathrm{yr}^{-1}]$")
pl.title("Merger rate density upper limit", fontsize=14.0)
pl.show()

### Redshift-time relation

The time since the Big Bang is given by:

$$t(z) = \int_z^\infty \frac{\mathrm{d}z'}{(1+z') H(z')}\,,$$

where

$$H(z') = \sqrt{\Omega_\Lambda + \Omega_m (1+z)^3}\,,$$

in a flat universe. We use the Planck 2015 values from [arXiv:1502.01589](https://arxiv.org/abs/1502.01589), noting that there is some tension between different measurements of $H_0$:

- $H_0 = 67.8 \,\,\mathrm{km/s}\,\mathrm{Mpc}^{-1}$
- $\Omega_\Lambda = 0.692$
- $\Omega_m = 0.308$

In [ ]:
#Get H in units of 1/yr
#(km/Mpc) = 3.24e-20
H0_peryr = 67.8*(3.24e-20)*(60*60*24*365)

Omega_L = 0.692
Omega_m = 0.308

def Hubble(z):
    return H0_peryr*np.sqrt(Omega_L + Omega_m*(1+z)**3)

In [ ]:
#Calculate time from Big Bang until redshift z (in years)
def t_univ(z):
    integ = lambda x: 1.0/((1+x)*Hubble(x))
    return quad(integ, z, np.inf)[0]

In [ ]:
#Plot age of the universe as a function of Redshift

zlist = np.linspace(0, 2, 50)
tvals = np.asarray([t_univ(z) for z in zlist])

pl.figure(figsize=(6.1,6))
pl.plot(zlist, tvals/1e9)

pl.xlabel(r"Redshift, $z$")
pl.ylabel(r"Time since Big Bang, $t$ [Gyr]")

pl.ylim(0, 15)

pl.show()

z_of_t = interp1d(tvals, zlist)

In [ ]:
#Now also plot z as a function of time

pl.figure(figsize=(6.1,6))
pl.plot(tvals/1e9,zlist)

pl.axhline(0.6, linestyle='--', color='k')

pl.ylabel(r"Redshift, $z$")
pl.xlabel(r"Time since Big Bang, $t$ [Gyr]")

pl.show()

### Merger Rate

Calculate Merger rate from PBH binaries, as a function of the merger time. This is given by:

$$ R_\mathrm{merge}(t_\mathrm{merge}) = n_\mathrm{PBH} P(t_\mathrm{merge})\,.$$

Here, $n_\mathrm{PBH}$ is the number density of PBHs and $P(t_\mathrm{merge})$ is the PDF for the merger time of the binaries. Note that not all PBHs will form binaries, so we choose to normalise $P(t_\mathrm{merge})$ to the number of binaries which form *per PBH*:

$$\int P(t_\mathrm{merge}) \,\mathrm{d}t_\mathrm{merge} = \frac{n_\mathrm{binaries}}{n_\mathrm{PBH}}\,.$$

From this, the maximum normalisation is obtained when all PBHs form binaries, i.e. $n_\mathrm{binaries}/n_\mathrm{PBH} = 1/2$.

***Mergers today***: We are interested in 'mergers today', by which we mean mergers roughly in the sensitivity range of LIGO. For some of the heaviest BHs, this could be out to redshift $z = 1$, so by 'today' we mean $z \in [0,1]$.

In [ ]:
#Calculate density of PBH binaries

#Some cosmological parameters
G_N = 4.302e-3 #(pc/solar mass) (km/s)^2

h = 0.678
H0 = 100.0*h #(km/s) Mpc^-1
Omega_DM = 0.1186/(h**2)
    
def calc_nPBH(f, M_PBH):

    #Fraction of DM made of PBHs
    Omega_PBH = f*Omega_DM
    rho = 3.0*H0**2/(8.0*np.pi*(G_N*1e-6)) #Solar masses per Mpc^3

    n_PBH = (1e3)**3*rho*Omega_PBH/M_PBH #PBH per Gpc^3
    
    return n_PBH
